In [60]:
import pandas as pd
import numpy as np
from math import inf
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from joblib import dump #more efficient than pickle on objects that carry large numpy arrays internally


In [61]:
df = pd.read_csv('../data/mnist/all_4hp_rusty.csv')
df = pd.read_csv('../data/mlpf/delphes_trainings_processed.csv')
df = df.iloc[:300]
df.head()

,bin_size,dropout,lr,n_glayers_id,n_glayers_reg,loss_0,loss_1,loss_2,loss_3,loss_4,...,loss_90,loss_91,loss_92,loss_93,loss_94,loss_95,loss_96,loss_97,loss_98,loss_99
0,16.0,0.006203,0.000141,4.0,1.0,533.897949,533.939148,533.939026,533.938904,533.938843,...,533.938782,533.938782,533.938782,533.938782,533.938782,533.938782,533.938782,533.938782,533.938782,533.938782
1,16.0,0.257000,0.000340,1.0,2.0,492.217072,471.338928,472.664246,473.120575,472.047882,...,474.869324,474.922974,474.983826,474.955231,475.082001,475.255493,492.020630,471.808075,472.234894,472.483948
2,64.0,0.302990,0.000250,2.0,0.0,529.882935,494.913788,492.469879,495.969910,498.249359,...,526.383667,526.397461,526.396484,526.405334,526.412476,526.418274,526.422729,526.416870,526.419739,526.418579
3,32.0,0.172210,0.000042,3.0,2.0,543.924561,484.485809,472.445312,467.696869,464.545593,...,448.215912,544.031372,484.386292,472.220398,467.484924,464.688690,462.955078,461.424591,460.127716,459.114685
4,64.0,0.422810,0.000220,3.0,1.0,507.416473,467.467712,465.895966,465.514557,465.184174,...,467.200775,466.911926,464.984131,464.936554,465.855988,465.208282,465.407593,465.519531,465.818481,466.210205


In [78]:
num_epochs = 100
known_curve = 0.25
min_hp_idx = 0
min_curve_idx = 6
max_hp_idx = 5 #when using 6hp search space
#max_hp_idx = 3  #when using 4hp search space

hps = df[df.columns[min_hp_idx:max_hp_idx+1]].to_numpy()

curve = df[df.columns[min_curve_idx:min_curve_idx+int(num_epochs*known_curve)]].to_numpy()
target = df[df.columns[min_curve_idx+num_epochs-2]].to_numpy()

In [79]:
#calculate finite diferences of 1st and 2nd order
def finite_difs(curve):
    difs1 = []
    for i in range(curve.shape[0]):
        difs1.append([])
        for j in range(1,curve.shape[1]):
            difs1[i].append(curve[i][j]-curve[i][j-1])
    difs2 = []
    for i in range(curve.shape[0]):
        difs2.append([])
        for j in range(1,len(difs1[0])):
            difs2[i].append(difs1[i][j]-difs1[i][j-1])
    difs1 = np.array(difs1)
    difs2 = np.array(difs2)
    return difs1, difs2

In [80]:
difs1, difs2 = finite_difs(curve)
X = np.append(np.append(np.append(hps,curve,1),difs1,1),difs2,1)
y = target

In [81]:
#scale
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.reshape(-1, 1))
#save the scalers so that they can be used when using the SVR in another program
dump(x_scaler,"x_scaler.joblib") 
dump(y_scaler,"y_scaler.joblib")

['y_scaler.joblib']

In [87]:
#split in train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(str(X_train.shape)+" "+str(X_test.shape))

(17, 78) (5, 78)


In [88]:
#instatiate and train predictor
model = NuSVR()
model.fit(X_train,y_train.ravel())

NuSVR()

In [ ]:
#Save model
filename = "modelo.joblib"
dump(model, "modelo.joblib")

In [89]:
#MSE
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred)

0.007594152785532956

In [90]:
#R^2
model.score(X_test,y_test)

0.9424735651718144

In [95]:
cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2')
cvs, cvs.mean(), cvs.std()

(array([0.98815634, 0.71272339, 0.93303573, 0.24733088, 0.86056278]),
 0.7483618249489711,
 0.2670194845487989)

In [887]:
#try to optimize regressor hps

from sklearn.model_selection import cross_val_score

best, C_best, Nu_best, gamma_best = -inf, -inf, -inf, -inf
for _ in range(1000):
    C = np.exp(np.random.uniform(np.log(1e-3),np.log(10.0)))
    Nu = np.random.uniform(0,1)
    gamma = "scale"
    model = NuSVR(C=C,nu=Nu,gamma=gamma)
    cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2').mean()
    if best < cvs:
        best = cvs
        C_best, Nu_best, gamma_best = C, Nu, gamma
best, C_best, Nu_best, gamma_best

(0.7663090107089143, 7.3135497213543115, 0.8924509605431109, 'scale')

In [888]:
model = NuSVR(C=C_best,nu=Nu_best,gamma=gamma_best)
model.fit(X_train,y_train.ravel())

cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2').mean()
cvs

0.7663090107089143

In [889]:
#model.fit(X_train,y_train.ravel())
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred), model.score(X_test,y_test)

(0.0005593760485826635, 0.9531359618257825)